In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sympy import (symbols, simplify)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
from sympy.physics.mechanics import Lagrangian, ReferenceFrame, Point, Particle,inertia, RigidBody, angular_momentum

In [ ]:
from optibot.symbolic import lagrange, diff_to_symb, SimpLagrangesMethod
from optibot.numpy import unpack

In [ ]:
init_vprinting()

In [ ]:
from optibot.robots import Panda_Simp

In [ ]:
panda = Panda_Simp()

In [ ]:
plt.style.use('default')

In [ ]:
from optibot.opti import Opti_Problem
import time

In [ ]:
t0 = time.time()
print('start time:',time.strftime('%H:%M:%S ', time.localtime(t0)))

for ii in range(1000000):
    _ = ii**2**0.9232


t1 = time.time()
print('end time:',time.strftime('%H:%M:%S ', time.localtime(t1)))
dt = t1-t0
print('dt: ',dt, time.strftime('%H:%M:%S ',time.gmtime(dt)))

#### Warning: next cell can take several minutes to run (the first time)

In [ ]:
t0 = time.time()
print('start time:',time.strftime('%H:%M:%S ', time.localtime(t0)))

N = 25
verbose = True
silent = False
ini_guess = 'lin'
t_end = 5
params = []
scheme = 'hs_mod'

_opti = Opti_Problem(
    LM=panda,
    params = params,
    scheme = scheme,
    ini_guess= ini_guess,
    t_end = t_end,
    verbose=verbose,
    silent=silent,
)
_opti.dynamic_setup()
_opti.opti_setup(N)

t1 = time.time()
print('end time:',time.strftime('%H:%M:%S ', time.localtime(t1)))
dt = t1-t0
print('dt: ',dt, time.strftime('%H:%M:%S ',time.gmtime(dt)))

In [ ]:
_opti.apply_scheme()

In [ ]:
panda.robot.qlim

In [ ]:
pose_0 = np.array([ 1.,  1.,  1., -1.,  1., 1.,  1. ])
pose_1 = np.array([-1., -1., -1., -2., -1., 1., -1. ])

In [ ]:
q_s = _opti.opti_points['q_s']
q_e = _opti.opti_points['q_e']
v_s = _opti.opti_points['v_s']
v_e = _opti.opti_points['v_e']

_opti.opti.subject_to(q_s == np.expand_dims(pose_0, 0))
_opti.opti.subject_to(q_e == np.expand_dims(pose_1, 0))

_opti.opti.subject_to(v_s == 0)
_opti.opti.subject_to(v_e == 0)

_opti.initial_guess(pose_0, pose_1)

In [ ]:
_opti.u_sq_cost()

In [ ]:
q_arr = _opti.opti_arrs['q']
q_arr_c = _opti.opti_arrs['q_c']
v_arr = _opti.opti_arrs['v']
v_arr_c = _opti.opti_arrs['v_c']
u_arr = _opti.opti_arrs['u']
u_arr_c = _opti.opti_arrs['u_c']
q_arr.shape

In [ ]:
panda.robot.qlim

In [ ]:
q_dot_max = np.zeros(7)
q_dot_max[0:4] = 2.1750
q_dot_max[4:7] = 2.6100
q_dot_max

In [ ]:
tau_max = np.zeros(7)
tau_max[0:4] = 87
tau_max[4:7] = 12
tau_max

In [ ]:
for ii in range(7):
    _opti.opti.subject_to(panda.robot.qlim[:,ii][0] <= q_arr[:,ii])
    _opti.opti.subject_to(panda.robot.qlim[:,ii][1] >= q_arr[:,ii])
    _opti.opti.subject_to(panda.robot.qlim[:,ii][0] <= q_arr_c[:,ii])
    _opti.opti.subject_to(panda.robot.qlim[:,ii][1] >= q_arr_c[:,ii])
    
    _opti.opti.subject_to(-q_dot_max[ii] <= v_arr[:,ii])
    _opti.opti.subject_to( q_dot_max[ii] >= v_arr[:,ii])
    _opti.opti.subject_to(-q_dot_max[ii] <= v_arr_c[:,ii])
    _opti.opti.subject_to( q_dot_max[ii] >= v_arr_c[:,ii])
    
    
    _opti.opti.subject_to(-tau_max[ii] <= u_arr[:,ii])
    _opti.opti.subject_to( tau_max[ii] >= u_arr[:,ii])
    _opti.opti.subject_to(-tau_max[ii] <= u_arr_c[:,ii])
    _opti.opti.subject_to( tau_max[ii] >= u_arr_c[:,ii])

In [ ]:
t0 = time.time()
print('start time:',time.strftime('%H:%M:%S ', time.localtime(t0)))

_opti.chrono_solve(1)

t1 = time.time()
print('end time:',time.strftime('%H:%M:%S ', time.localtime(t1)))
dt = t1-t0
print('dt: ',dt, time.strftime('%H:%M:%S ',time.gmtime(dt)))

In [ ]:
_opti.results['cpudt'],time.strftime('%H:%M:%S ',time.gmtime(_opti.results['cpudt']))

In [ ]:
%matplotlib inline

In [ ]:
plt.figure(figsize=[16,10])
plt.plot(_opti.results['t'],_opti.results['q'],marker='o')
plt.plot(_opti.results['t_c'],_opti.results['q_c'],'o')
plt.grid()

In [ ]:
plt.figure(figsize=[16,10])
plt.plot(_opti.results['t'],_opti.results['v'],marker='o')
plt.grid()

In [ ]:
plt.figure(figsize=[16,10])
plt.plot(_opti.results['t'],_opti.results['u'],marker='o')
plt.plot(_opti.results['t_c'],_opti.results['u_c'],'o')
plt.grid()

In [ ]:
_opti.N

In [ ]:
fname = f'panda_opti_ej_{_opti.scheme}_{_opti.N}.txt'
f = open(fname, "w")
for ii in _opti.results:
    f.write(f'\n\n\t\t{ii}\n')
    f.write(str(_opti.results[ii]))
f.close()

In [ ]:
1+1

In [ ]:
if _opti.scheme == 'hs':
    poly_deg = 3
elif _opti.scheme == 'hs_mod':
    poly_deg = 4
coefmatrix = np.zeros([_opti.N+1, 1+_opti.n_q*2*(poly_deg+1)])
coefmatrix.shape

In [ ]:
coefmatrix[:,0] = _opti.results['t']

In [ ]:
_x = _opti.results['x']
_x_d = _opti.results['x_d']
_x_c = _opti.results['x_c']
_x_d_c = _opti.results['x_d_c']

_q = _opti.results['q']
_v = _opti.results['v']
_q_c = _opti.results['q_c']
_v_c = _opti.results['v_c']
_a = _opti.results['a']
_a_c = _opti.results['a_c']

In [ ]:
_opti.scheme

In [ ]:
h = _opti.results['t'][1:] - _opti.results['t'][:-1]
h = np.expand_dims(h,1)

if _opti.scheme == 'hs':
    _a = _x
    _b = _x_d[:-1,:]
    _c = -(3*_b - 4*_x_d_c + _x_d[1:,:])/(2*h)
    _d = (2*_b - 4*_x_d_c + 2*_x_d[1:,:])/(3*h*h)
    
    coefmatrix[:,1::4] = _a
    coefmatrix[:-1,2::4] = _b
    coefmatrix[:-1,3::4] = _c
    coefmatrix[:-1,4::4] = _d

elif _opti.scheme == 'hs_mod':
    
    _aa = _q
    _b = _v[:-1,:]
    _c = _a[:-1,:]/2
    _d = -(3*_a[:-1,:] - 4*_a_c + _a[1:,:])/(6*h)
    _e = (2*_a[:-1,:] - 4*_a_c + 2*_a[1:,:])/(12*h*h)
    
    n = 1+_opti.n_q*(poly_deg+1)
    
    coefmatrix[:,1:n:5] = _aa
    coefmatrix[:-1,2:n:5] = _b
    coefmatrix[:-1,3:n:5] = _c
    coefmatrix[:-1,4:n:5] = _d
    coefmatrix[:-1,5:n:5] = _e
    
    coefmatrix[:-1,n::5] = _b
    coefmatrix[:-1,n+1::5] = _a[:-1,:]
    coefmatrix[:-1,n+2::5] = _d*3
    coefmatrix[:-1,n+3::5] = _e*4
    coefmatrix[:-1,n+4::5] = 0

In [ ]:
coefmatrix[:-1,n::5].shape

In [ ]:
fname = f'panda_coef_opti_ej_{_opti.scheme}_{_opti.N}.txt'
np.savetxt(
    fname,
    coefmatrix,
    '%1.3f',
    comments='',
    header=f'{_opti.N}\n{poly_deg}',
)

In [ ]:
_XX = coefmatrix[:,1:]

In [ ]:
_XX.shape

In [ ]:
t0 = time.time()
print('start time:',time.strftime('%H:%M:%S ', time.localtime(t0)))

N = 25
verbose = True
silent = False
ini_guess = 'lin'
t_end = 5
params = []
scheme = 'hs_mod'

_opti_2 = Opti_Problem(
    LM=panda,
    params = params,
    scheme = scheme,
    ini_guess= ini_guess,
    t_end = t_end,
    verbose=verbose,
    silent=silent,
)
_opti_2.dynamic_setup()
_opti_2.opti_setup(N)

t1 = time.time()
print('end time:',time.strftime('%H:%M:%S ', time.localtime(t1)))
dt = t1-t0
print('dt: ',dt, time.strftime('%H:%M:%S ',time.gmtime(dt)))

In [ ]:
_opti_2.apply_scheme()

In [ ]:
q_s = _opti_2.opti_points['q_s']
q_e = _opti_2.opti_points['q_e']
v_s = _opti_2.opti_points['v_s']
v_e = _opti_2.opti_points['v_e']

_opti_2.opti.subject_to(q_s == np.expand_dims(pose_0, 0))
_opti_2.opti.subject_to(q_e == np.expand_dims(pose_1, 0))

_opti_2.opti.subject_to(v_s == 0)
_opti_2.opti.subject_to(v_e == 0)

_opti_2.initial_guess(pose_0, pose_1)

_opti_2.u_sq_cost()

q_arr = _opti_2.opti_arrs['q']
q_arr_c = _opti_2.opti_arrs['q_c']
v_arr = _opti_2.opti_arrs['v']
v_arr_c = _opti_2.opti_arrs['v_c']
u_arr = _opti_2.opti_arrs['u']
u_arr_c = _opti_2.opti_arrs['u_c']
q_arr.shape

for ii in range(7):
    _opti_2.opti.subject_to(panda.robot.qlim[:,ii][0] <= q_arr[:,ii])
    _opti_2.opti.subject_to(panda.robot.qlim[:,ii][1] >= q_arr[:,ii])
    _opti_2.opti.subject_to(panda.robot.qlim[:,ii][0] <= q_arr_c[:,ii])
    _opti_2.opti.subject_to(panda.robot.qlim[:,ii][1] >= q_arr_c[:,ii])
    
    _opti_2.opti.subject_to(-q_dot_max[ii] <= v_arr[:,ii])
    _opti_2.opti.subject_to( q_dot_max[ii] >= v_arr[:,ii])
    _opti_2.opti.subject_to(-q_dot_max[ii] <= v_arr_c[:,ii])
    _opti_2.opti.subject_to( q_dot_max[ii] >= v_arr_c[:,ii])
    
    
    _opti_2.opti.subject_to(-tau_max[ii] <= u_arr[:,ii])
    _opti_2.opti.subject_to( tau_max[ii] >= u_arr[:,ii])
    _opti_2.opti.subject_to(-tau_max[ii] <= u_arr_c[:,ii])
    _opti_2.opti.subject_to( tau_max[ii] >= u_arr_c[:,ii])

In [ ]:
t0 = time.time()
print('start time:',time.strftime('%H:%M:%S ', time.localtime(t0)))

_opti_2.chrono_solve(1)

t1 = time.time()
print('end time:',time.strftime('%H:%M:%S ', time.localtime(t1)))
dt = t1-t0
print('dt: ',dt, time.strftime('%H:%M:%S ',time.gmtime(dt)))

In [ ]:
plt.figure(figsize=[16,10])
plt.plot(_opti_2.results['t'],_opti_2.results['q'],marker='o')
plt.plot(_opti_2.results['t_c'],_opti_2.results['q_c'],'o')
plt.grid()

plt.figure(figsize=[16,10])
plt.plot(_opti_2.results['t'],_opti_2.results['u'],marker='o')
plt.plot(_opti_2.results['t_c'],_opti_2.results['u_c'],'o')
plt.grid()

In [ ]:
from roboticstoolbox.models.DH import Panda as PD

In [ ]:
pd = PD()

In [ ]:
pd

In [ ]:
pd.fkine(_opti.results['q'])

In [ ]:
%matplotlib notebook
%config NotebookBackend.figure_format = 'retina'

In [ ]:
pd.plot(_opti.results['q'])